# Chapter 23: Type Inference

---

## 23.1 How Type Inference Works

Type inference is TypeScript's ability to automatically deduce types from the code without explicit type annotations. This mechanism reduces verbosity while maintaining type safety, allowing developers to write cleaner code without sacrificing the benefits of static typing.

### 23.1.1 The Inference Engine

TypeScript uses a sophisticated constraint-based type inference system that analyzes the structure and flow of values through your code.

```typescript
// Basic inference from initializers
let message = "Hello TypeScript";  // Inferred as string
const count = 42;                  // Inferred as 42 (literal type)
let isActive = true;               // Inferred as boolean

// TypeScript analyzes the right-hand side and assigns the most specific type
const user = {
  id: 1,
  name: "John",
  isAdmin: false
};
// Inferred as: { id: number; name: string; isAdmin: boolean; }

// Array inference
const numbers = [1, 2, 3];  
// Inferred as number[] (not [1, 2, 3] because let allows mutation)

const tuple = [1, "hello"] as const;
// Inferred as readonly [1, "hello"] (tuple with literal types)
```

**The Inference Process:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    TypeScript Inference Process                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. Value Analysis                                                 │
│      ┌─────────────────┐                                            │
│      │ const x = "abc" │ ──► Analyze value "abc"                   │
│      └─────────────────┘                                            │
│                                                                     │
│   2. Type Deduction                                                 │
│      • Primitives: Most specific literal type for const             │
│      • Objects: Structural type with inferred property types        │
│      • Arrays: Element type union or array type                     │
│      • Functions: Parameter and return type inference               │
│                                                                     │
│   3. Constraint Checking                                            │
│      ┌──────────────────────────────────────┐                       │
│      │ Check compatibility with usage sites │                       │
│      │ Widen literal types if necessary     │                       │
│      └──────────────────────────────────────┘                       │
│                                                                     │
│   4. Contextual Refinement                                            │
│      • Use surrounding context to narrow types                      │
│      • Apply control flow analysis                                    │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 23.1.2 Explicit vs Implicit Typing

Understanding when to rely on inference versus explicit annotations is crucial for maintainable code.

```typescript
// ✅ Good: Let TypeScript infer simple types
const name = "John";                    // string
const age = 30;                         // number
const isActive = true;                  // boolean
const items = [1, 2, 3];                // number[]

// ✅ Good: Explicit types for public APIs
interface UserService {
  getUser(id: number): Promise<User>;   // Explicit return type
  saveUser(user: User): Promise<void>;  // Clear contract
}

// ✅ Good: Explicit types when inference would be too wide
const config: {                         // Explicit object shape
  apiUrl: string;
  timeout: number;
} = {
  apiUrl: "https://api.example.com",
  timeout: 5000
};

// ❌ Bad: Unnecessary explicit annotations (verbose)
const userName: string = "John";
const userAge: number = 30;
const isUserActive: boolean = true;

// ❌ Bad: Missing explicit types where they add clarity
function processData(data) {            // Implicit any (if noImplicitAny is off)
  // Implementation
}

// ✅ Good: Complex return types benefit from explicit annotations
function createUser(data: UserFormData): User {
  // TypeScript checks that return matches User interface
  return {
    id: generateId(),
    ...data,
    createdAt: new Date()
  };
}
```

**Inference Guidelines:**

```typescript
// When to use inference:
// 1. Local variables with obvious types
const localValue = calculateTotal();    // Return type inferred from function

// 2. Callback functions (contextual typing)
items.map(item => item * 2);            // Parameter type inferred from array

// 3. Generic type arguments (often inferred)
const result = Promise.resolve(42);     // Promise<number> inferred

// When to use explicit types:
// 1. Function return types for public APIs
export function fetchUser(id: number): Promise<User> {
  return api.get(`/users/${id}`);
}

// 2. Object literals that might be widened
const statusCodes: Record<string, number> = {
  ok: 200,
  notFound: 404
};

// 3. Places where you want literal types preserved
type HttpMethod = "GET" | "POST" | "PUT" | "DELETE";
const method: HttpMethod = "GET";       // Without annotation, inferred as string
```

---

## 23.2 Best Common Type Algorithm

When TypeScript encounters multiple types in a single expression (like array literals or ternary operators), it uses the "Best Common Type" algorithm to find a suitable supertype that encompasses all possibilities.

### 23.2.1 Finding the Common Supertype

TypeScript looks for the most specific type that is compatible with all candidates.

```typescript
// Array literal with multiple types
const mixed = [1, 2, "three"];
// Best Common Type: (string | number)[]
// Not [number, number, string] because arrays are mutable

// Ternary operator
const value = Math.random() > 0.5 ? "hello" : 42;
// Best Common Type: string | number

// Function returns
function getValue(condition: boolean) {
  if (condition) {
    return "string value";
  }
  return 42;
}
// Return type inferred as: string | number

// Object literals
const users = [
  { name: "John", age: 30 },
  { name: "Jane", email: "jane@example.com" }
];
// Best Common Type: { name: string; age?: number; email?: string; }[]
// Properties present in some but not all become optional
```

**How Best Common Type Works:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                Best Common Type Algorithm                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Input: [1, 2, "three"]                                            │
│                                                                     │
│   Step 1: Identify candidate types                                  │
│   • 1: numeric literal 1                                            │
│   • 2: numeric literal 2                                            │
│   • "three": string literal "three"                                 │
│                                                                     │
│   Step 2: Find common supertype                                     │
│   • 1 and 2 share supertype: number                                 │
│   • number and string share supertype: string | number              │
│                                                                     │
│   Step 3: Apply array wrapper                                       │
│   • Result: (string | number)[]                                     │
│                                                                     │
│   Alternative with 'as const':                                      │
│   [1, 2, "three"] as const → readonly [1, 2, "three"]               │
│   (Preserves literal types as tuple)                                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 23.2.2 Structural Compatibility in Union Types

When combining object types, TypeScript creates a union of properties that accounts for all shapes.

```typescript
// Union of different object shapes
const shapes = [
  { kind: "circle", radius: 10 },
  { kind: "square", side: 10 },
  { kind: "rectangle", width: 10, height: 20 }
];

// Inferred type:
// ({
//   kind: "circle";
//   radius: number;
//   side?: undefined;
//   width?: undefined;
//   height?: undefined;
// } | {
//   kind: "square";
//   radius?: undefined;
//   side: number;
//   width?: undefined;
//   height?: undefined;
// } | {
//   kind: "rectangle";
//   radius?: undefined;
//   side?: undefined;
//   width: number;
//   height: number;
// })[]

// Better approach: Explicit discriminated union
type Shape = 
  | { kind: "circle"; radius: number }
  | { kind: "square"; side: number }
  | { kind: "rectangle"; width: number; height: number };

const typedShapes: Shape[] = [
  { kind: "circle", radius: 10 },
  { kind: "square", side: 10 }
];
```

### 23.2.3 Handling Empty Arrays and Objects

TypeScript handles empty collections carefully to allow subsequent operations while maintaining safety.

```typescript
// Empty array inference
const empty = [];           // Inferred as any[] (with strictNullChecks: never[])
empty.push(1);              // Now number[]
empty.push("hello");        // Now (string | number)[]

// With explicit type annotation
const numbers: number[] = [];  // Stays number[]
numbers.push(1);               // OK
// numbers.push("hello");      // ❌ Error

// Empty object
const emptyObj = {};        // Inferred as {}
// emptyObj.name = "John";   // ❌ Error: property doesn't exist

// Better: Use Record or interface
const record: Record<string, string> = {};
record.name = "John";       // ✅ OK
```

---

## 23.3 Return Type Inference

TypeScript infers function return types by analyzing all return statements and applying the Best Common Type algorithm.

### 23.3.1 Explicit vs Inferred Return Types

While TypeScript can infer return types, explicit annotations provide documentation and catch implementation errors.

```typescript
// Inferred return type
function greet(name: string) {
  return `Hello, ${name}`;
}
// Inferred: string

// Explicit return type (recommended for public functions)
function greetExplicit(name: string): string {
  return `Hello, ${name}`;
}

// Catching errors with explicit return types
function getUser(id: number): User {
  return {
    id: id,
    name: "John",
    // email: "john@example.com"  // ❌ Error: missing email if required in User
  };
}

// Async function inference
async function fetchData() {
  return { data: "value" };
}
// Inferred: Promise<{ data: string; }>

// Explicit Promise return type
async function fetchUser(id: number): Promise<User> {
  const response = await api.get(`/users/${id}`);
  return response.data;  // TypeScript checks this matches User
}
```

### 23.3.2 Multiple Return Paths

When a function has multiple return statements, TypeScript unions all possible return types.

```typescript
// Multiple return types
function processValue(value: string | number) {
  if (typeof value === "string") {
    return value.toUpperCase();    // Returns string
  }
  return value * 2;                // Returns number
}
// Inferred return type: string | number

// Early returns
function validateUser(user: User | null) {
  if (!user) {
    return null;                   // null
  }
  if (!user.email) {
    return undefined;              // undefined
  }
  return user;                     // User
}
// Inferred: User | null | undefined

// Using type guards to narrow
function getUserName(user: User | null): string {
  if (!user) {
    return "Anonymous";
  }
  return user.name;
}
// Inferred: string (unified return type)
```

### 23.3.3 Recursive Function Inference

Recursive functions require careful handling as TypeScript must determine the return type from potentially circular references.

```typescript
// Factorial with inferred return type
function factorial(n: number) {
  if (n <= 1) return 1;
  return n * factorial(n - 1);
}
// Inferred: number

// Recursive tree traversal
interface TreeNode {
  value: number;
  children?: TreeNode[];
}

function sumTree(node: TreeNode): number {
  let sum = node.value;
  if (node.children) {
    sum += node.children.reduce((acc, child) => acc + sumTree(child), 0);
  }
  return sum;
}
// Explicit return type helps TypeScript verify correctness

// Mutually recursive functions
function isEven(n: number): boolean {
  if (n === 0) return true;
  return isOdd(n - 1);
}

function isOdd(n: number): boolean {
  if (n === 0) return false;
  return isEven(n - 1);
}
// TypeScript infers both return types as boolean
```

---

## 23.4 Contextual Typing

Contextual typing occurs when the type of an expression is implied by its location, allowing TypeScript to infer types for function parameters and object properties based on their usage context.

### 23.4.1 Callback Parameter Inference

When functions are passed as callbacks, TypeScript infers parameter types from the expected signature.

```typescript
// Array methods with contextual typing
const numbers = [1, 2, 3, 4, 5];

// TypeScript knows 'n' is number from the array type
const doubled = numbers.map(n => n * 2);

// TypeScript knows 'acc' is number and 'curr' is number
const sum = numbers.reduce((acc, curr) => acc + curr, 0);

// Event handlers with contextual typing
const button = document.querySelector("button");

button?.addEventListener("click", event => {
  // 'event' is contextually typed as MouseEvent
  console.log(event.clientX, event.clientY);
});

// Generic function contextual typing
function processArray<T>(items: T[], processor: (item: T) => string): string[] {
  return items.map(processor);
}

// TypeScript infers T as number, so 'item' is number
const strings = processArray([1, 2, 3], item => item.toFixed(2));
```

### 23.4.2 Object Property Context

Object literals receive contextual types when assigned to typed variables or passed as arguments.

```typescript
// Interface providing context
interface Config {
  host: string;
  port: number;
  ssl?: boolean;
}

// Contextual typing on object literal
const config: Config = {
  host: "localhost",
  port: 3000,
  // ssl: "true"  // ❌ Error: TypeScript knows ssl should be boolean
};

// Function argument context
function setupServer(config: Config) {
  // Implementation
}

setupServer({
  host: "localhost",
  port: 3000
  // TypeScript provides autocomplete and type checking
});

// Contextual typing with union types
type Action = 
  | { type: "increment"; amount: number }
  | { type: "decrement"; amount: number };

function dispatch(action: Action) {
  // Implementation
}

dispatch({
  type: "increment",
  amount: 5
  // TypeScript knows this must match one of the Action variants
});
```

### 23.4.3 Tuple Contextual Typing

Tuples provide strict contextual typing for array literals.

```typescript
// Tuple type provides context
type Point = [number, number];

const p1: Point = [10, 20];        // ✅ OK
// const p2: Point = [10, "20"];   // ❌ Error: second element must be number
// const p3: Point = [10];         // ❌ Error: missing second element
// const p4: Point = [10, 20, 30]; // ❌ Error: too many elements

// Function returning tuple
function getCoordinates(): [number, number] {
  return [0, 0];  // Contextually typed as tuple, not number[]
}

// Destructuring with contextual typing
function useState<T>(initial: T): [T, (newValue: T) => void] {
  return [initial, () => {}];
}

const [count, setCount] = useState(0);
// count is inferred as number
// setCount is inferred as (newValue: number) => void
```

---

## 23.5 Type Inference in Arrays

Array inference balances specificity with flexibility, considering whether arrays are mutable and how they're initialized.

### 23.5.1 Mutable vs Immutable Inference

TypeScript infers different array types based on mutability expectations and the `const` assertion.

```typescript
// Mutable array (default)
const mutable = [1, 2, 3];
// Type: number[]
// Can push any number
mutable.push(4);        // ✅ OK

// Const assertion (immutable tuple)
const immutable = [1, 2, 3] as const;
// Type: readonly [1, 2, 3]
// immutable.push(4);   // ❌ Error: readonly array

// Mixed types without const assertion
const mixed = [1, "two", 3];
// Type: (string | number)[]
// Widened to union for flexibility

// Mixed types with const assertion
const mixedConst = [1, "two", 3] as const;
// Type: readonly [1, "two", 3]
// Preserves literal types and order

// Accessing const array elements
const first = mixedConst[0];   // Type: 1 (literal)
const second = mixedConst[1];  // Type: "two" (literal)
```

### 23.5.2 Empty Array Handling

Empty arrays receive special treatment to allow gradual type building.

```typescript
// Empty array initialization
const empty = [];           // Type: any[] (or never[] with strict settings)

// Type evolves with pushes
empty.push(1);              // Type becomes number[]
empty.push("hello");        // Type becomes (string | number)[]

// Explicit type preferred for empty arrays
const numbers: number[] = [];
numbers.push(1);            // ✅ OK
// numbers.push("hello");   // ❌ Error

// Using generic Array type
const strings: Array<string> = [];

// Readonly arrays
const readonlyNumbers: readonly number[] = [1, 2, 3];
// readonlyNumbers.push(4); // ❌ Error
```

### 23.5.3 Array Method Return Types

Array methods infer return types based on the callback return types and array element types.

```typescript
const numbers = [1, 2, 3, 4, 5];

// Map infers from callback return
const strings = numbers.map(n => n.toString());
// Type: string[]

// Filter with type predicate
const evens = numbers.filter((n): n is number => n % 2 === 0);
// Type: number[] (same as input)

// Filter without predicate (narrowing)
const maybeNumbers = [1, "two", 3, "four"];
const justNumbers = maybeNumbers.filter((x): x is number => typeof x === "number");
// Type: number[]

// Reduce infers accumulator from initial value
const sum = numbers.reduce((acc, n) => acc + n, 0);
// Type: number

// Reduce with different accumulator type
const indexed = numbers.reduce((acc, n, i) => {
  acc[n] = i;
  return acc;
}, {} as Record<number, number>);
// Type: Record<number, number>
```

---

## 23.6 Inferring in Conditional Types

The `infer` keyword in conditional types creates a powerful mechanism for extracting and manipulating types within type definitions.

### 23.6.1 Basic Inference Patterns

Use `infer` to capture types from complex structures without explicit declaration.

```typescript
// Extract return type
type ReturnType<T> = T extends (...args: any[]) => infer R ? R : never;

function getUser() {
  return { id: 1, name: "John" };
}

type UserReturn = ReturnType<typeof getUser>;
// { id: number; name: string; }

// Extract promise resolution type
type Awaited<T> = T extends Promise<infer U> ? U : T;

async function fetchData() {
  return "data";
}

type Fetched = Awaited<ReturnType<typeof fetchData>>;
// string

// Extract array element type
type ElementType<T> = T extends (infer E)[] ? E : never;

const items = [1, 2, 3];
type Item = ElementType<typeof items>;  // number
```

### 23.6.2 Multiple Inference Points

Extract multiple related types simultaneously from a single structure.

```typescript
// Extract both key and value types from an entry
type Entry<T> = T extends [infer K, infer V] ? { key: K; value: V } : never;

const entry: ["id", number] = ["id", 1];
type Parsed = Entry<typeof entry>;  // { key: "id"; value: number; }

// Extract function parameters and return type
type FunctionInfo<T> = T extends (...args: infer P) => infer R
  ? { params: P; return: R }
  : never;

function example(a: string, b: number): boolean {
  return true;
}

type Info = FunctionInfo<typeof example>;
// { params: [a: string, b: number]; return: boolean; }

// Extract constructor parameters and instance type
type ClassInfo<T> = T extends new (...args: infer P) => infer I
  ? { constructorParams: P; instance: I }
  : never;

class User {
  constructor(name: string, age: number) {}
}

type UserClassInfo = ClassInfo<typeof User>;
// { constructorParams: [name: string, age: number]; instance: User; }
```

### 23.6.3 Recursive Type Inference

Combine `infer` with recursive conditional types for deep type extraction.

```typescript
// Deep promise unwrapping
type DeepAwaited<T> = T extends Promise<infer U>
  ? DeepAwaited<U>
  : T;

type NestedPromise = Promise<Promise<Promise<string>>>;
type Unwrapped = DeepAwaited<NestedPromise>;  // string

// Extract all function return types in a chain
type ChainReturn<T> = T extends (arg: any) => infer R
  ? R extends (arg: any) => any
    ? ChainReturn<R>
    : R
  : T;

const chain = (x: number) => (y: string) => (z: boolean) => ({ x, y, z });
type Final = ChainReturn<typeof chain>;  // { x: number; y: string; z: boolean; }

// Infer deepest array element type
type DeepElement<T> = T extends (infer U)[]
  ? DeepElement<U>
  : T;

type DeepArray = number[][][];
type Deepest = DeepElement<DeepArray>;  // number
```

---

## 23.7 Controlling Inference

Sometimes TypeScript's default inference needs guidance. Several techniques help control and constrain type inference.

### 23.7.1 Const Assertions

The `as const` assertion prevents type widening and preserves literal types.

```typescript
// Without const assertion (widening)
const config = {
  apiUrl: "https://api.example.com",
  timeout: 5000
};
// Type: { apiUrl: string; timeout: number; }

// With const assertion (preservation)
const configConst = {
  apiUrl: "https://api.example.com",
  timeout: 5000
} as const;
// Type: {
//   readonly apiUrl: "https://api.example.com";
//   readonly timeout: 5000;
// }

// Useful for discriminated unions
const actions = [
  { type: "increment", amount: 1 },
  { type: "decrement", amount: 1 }
] as const;

// Type: readonly [
//   { readonly type: "increment"; readonly amount: 1; },
//   { readonly type: "decrement"; readonly amount: 1; }
// ]

type Action = (typeof actions)[number];
// { type: "increment"; amount: 1; } | { type: "decrement"; amount: 1; }

// Const assertion with arrays
const methods = ["GET", "POST", "PUT"] as const;
type HttpMethod = (typeof methods)[number];  // "GET" | "POST" | "PUT"
```

### 23.7.2 Type Parameter Constraints

Constrain generic type parameters to guide inference toward specific type shapes.

```typescript
// Constrained generic
function processRecord<K extends string, V>(
  record: Record<K, V>
): [K, V][] {
  return Object.entries(record) as [K, V][];
}

// TypeScript infers K as specific literals, not just string
const result = processRecord({ a: 1, b: 2 });
// Type: [("a" | "b"), number][]

// Without constraint, inference is less specific
function looseProcess<K, V>(record: Record<K, V>): [K, V][] {
  return Object.entries(record) as [K, V][];
}

const looseResult = looseProcess({ a: 1, b: 2 });
// Type: [string, number][] (less specific)

// Multiple constraints
interface HasId {
  id: number;
}

function updateById<T extends HasId>(
  items: T[],
  id: number,
  update: Partial<T>
): T[] {
  return items.map(item => 
    item.id === id ? { ...item, ...update } : item
  );
}
```

### 23.7.3 Explicit Type Arguments

Sometimes you need to explicitly specify generic type arguments when inference fails or produces undesired types.

```typescript
// Generic function with multiple type parameters
function createPair<T, U>(first: T, second: U): [T, U] {
  return [first, second];
}

// Explicit type arguments
const pair = createPair<string, number>("hello", 42);
// Explicitly typed as [string, number]

// Without explicit types (inferred)
const inferred = createPair("hello", 42);
// Inferred as [string, number] (same in this case)

// When inference needs help
function fetchData<T>(url: string): Promise<T> {
  return fetch(url).then(r => r.json());
}

// Without explicit type, T is inferred as unknown
// const data = await fetchData("/api/user");

// With explicit type
const user = await fetchData<User>("/api/user");
// Type: User

// Partial type arguments (TypeScript 4.7+)
function createMap<K extends string, V>(entries: [K, V][]): Map<K, V> {
  return new Map(entries);
}

// Can specify just K, let V be inferred
const map = createMap<"a" | "b">([["a", 1], ["b", 2]]);
// Type: Map<"a" | "b", number>
```

### 23.7.4 Satisfies Operator

The `satisfies` operator (TypeScript 4.9+) checks that an expression matches a type without widening it.

```typescript
// Without satisfies (widening occurs)
const config = {
  host: "localhost",
  port: 3000
};
// Type: { host: string; port: number; }

// With satisfies (checking without widening)
const configSatisfies = {
  host: "localhost",
  port: 3000
} satisfies { host: string; port: number };

// Type is inferred as { host: "localhost"; port: 3000; }
// But checked against { host: string; port: number; }

// Catching excess properties with satisfies
const strictConfig = {
  host: "localhost",
  port: 3000,
  // protocol: "https"  // ❌ Error if not in the satisfies type
} satisfies { host: string; port: number };

// Combining with const assertion behavior
const preciseConfig = {
  host: "localhost",
  port: 3000
} as const satisfies { host: string; port: number };
// Type: { readonly host: "localhost"; readonly port: 3000; }
```

---

## 23.8 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored TypeScript's type inference mechanisms:

**Key Takeaways:**

1. **Inference Basics**:
   - TypeScript automatically deduces types from values
   - Uses constraint-based analysis for flexible yet safe typing
   - Balances specificity with practicality

2. **Best Common Type**:
   - Algorithm finds the most specific supertype for unions
   - Handles object property unification
   - Manages empty collections specially

3. **Return Type Inference**:
   - Analyzes all return paths in functions
   - Unions multiple return types
   - Benefits from explicit annotations in public APIs

4. **Contextual Typing**:
   - Types inferred from usage context (callbacks, object literals)
   - Parameter types inferred from function signatures
   - Tuple types provide strict contextual checking

5. **Array Inference**:
   - Mutable arrays widen to element type
   - `as const` preserves literal types and creates tuples
   - Empty arrays start as flexible `any[]` or `never[]`

6. **Conditional Type Inference**:
   - `infer` keyword extracts types from patterns
   - Supports multiple inference points
   - Enables recursive type extraction

7. **Controlling Inference**:
   - `as const` prevents widening
   - Constraints guide generic inference
   - Explicit type arguments override inference
   - `satisfies` checks types without widening

### Practical Exercises

**Exercise 1: Basic Inference**

Predict the inferred types:

```typescript
// 1. What is the inferred type of 'message'?
const message = "Hello TypeScript";

// 2. What is the inferred type of 'config'?
const config = {
  host: "localhost",
  port: 3000,
  ssl: true
};

// 3. What is the inferred type of 'mixed'?
const mixed = [1, "two", 3];

// 4. What is the inferred type of 'tuple'?
const tuple = [1, "two", 3] as const;

// 5. What is the return type of 'process'?
function process(x: string | number) {
  if (typeof x === "string") return x.length;
  return x * 2;
}

// 6. What is the inferred type of 'result'?
const result = [1, 2, 3].map(n => n.toString());
```

**Exercise 2: Contextual Typing**

Fix the typing issues using contextual typing:

```typescript
// 1. Make this type-safe without explicit parameter types
const items = [1, 2, 3];
const processed = items.map((x) => x.toFixed(2));

// 2. Fix the event handler typing
const button = document.querySelector("button");
button?.addEventListener("click", (e) => {
  console.log(e.clientX);
});

// 3. Type this reducer correctly
const actions = [
  { type: "increment", amount: 1 },
  { type: "decrement", amount: 1 }
] as const;

function reducer(state: number, action: /* derive from actions */) {
  switch (action.type) {
    case "increment": return state + action.amount;
    case "decrement": return state - action.amount;
  }
}
```

**Exercise 3: Advanced Inference**

Implement these type utilities using inference:

```typescript
// 1. Implement a type that extracts the resolved type of a Promise
type MyAwaited<T> = /* your code */;

// 2. Implement a type that extracts the first element of a tuple
type First<T extends any[]> = /* your code */;

// 3. Implement a type that extracts the last element of a tuple
type Last<T extends any[]> = /* your code */;

// 4. Implement a type that counts the length of a tuple
type Length<T extends any[]> = /* your code */;

// 5. Implement a type that checks if a type is never
type IsNever<T> = /* your code */;
```

**Exercise 4: Real-World Inference**

Build a type-safe API client with proper inference:

```typescript
// Create a function that:
// 1. Accepts a URL and options object
// 2. Infers the return type from a generic parameter
// 3. Uses const assertions for method literals
// 4. Provides autocomplete for headers

declare function apiClient<T>(
  url: string,
  options: {
    method: "GET" | "POST" | "PUT" | "DELETE";
    headers?: Record<string, string>;
    body?: unknown;
  }
): Promise<T>;

// Usage should infer types correctly:
const user = await apiClient<User>("/api/user", {
  method: "GET",  // Should autocomplete and validate
  headers: {
    "Content-Type": "application/json"  // Should allow any string key
  }
});
```

**Exercise 5: Inference Control**

Use `as const` and `satisfies` appropriately:

```typescript
// 1. Create a configuration object where:
//    - Keys are exactly "host", "port", "ssl"
//    - Values preserve their literal types (not widened to string/number/boolean)
//    - TypeScript validates the shape matches an interface

const config = /* your code */ {
  host: "localhost",
  port: 3000,
  ssl: true
};

// 2. Create an array of actions where:
//    - Each action has a specific literal type
//    - The array is readonly
//    - You can extract a union type of all actions

const actions = /* your code */ [
  { type: "login", payload: { username: string } },
  { type: "logout", payload: {} }
];

// 3. Create a function that accepts the actions above and:
//    - Narrows the type based on the action type
//    - Has exhaustiveness checking

function handleAction(action: /* derive from actions */) {
  // Implementation with type narrowing
}
```

### Additional Resources

- **TypeScript Handbook - Type Inference**: https://www.typescriptlang.org/docs/handbook/type-inference.html
- **TypeScript Handbook - Advanced Types**: https://www.typescriptlang.org/docs/handbook/advanced-types.html
- **TypeScript Deep Dive - Type Inference**: https://basarat.gitbook.io/typescript/type-system/type-inference
- **Total TypeScript - Inference Tips**: https://www.totaltypescript.com/tips/infer

---

## Coming Up Next: Chapter 24 - Declaration Merging

In the next chapter, we will explore **Declaration Merging**, TypeScript's unique ability to combine multiple declarations with the same name into a single definition:

- **24.1 Understanding Declaration Merging** - How TypeScript combines declarations
- **24.2 Merging Interfaces** - Augmenting interface definitions
- **24.3 Merging Namespaces** - Extending namespace contents
- **24.4 Merging Namespaces with Classes** - Pattern for static members
- **24.5 Merging Namespaces with Functions** - Adding properties to functions
- **24.6 Merging Namespaces with Enums** - Extending enum functionality
- **24.7 Module Augmentation** - Extending external module types

Declaration merging is a powerful feature that enables progressive type definition, third-party library augmentation, and sophisticated design patterns that bridge the gap between value and type spaces in TypeScript.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='22. the_keyof_operator.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='24. declaration_merging.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
